In [2]:
import requests
import json
import pandas as pd
from config import covid_api
from config import google_api
import gmaps

# NOTES:
# https://api.covid19api.com/world/total
# This is all data - returns 10MB+
# https://api.covid19api.com/all


In [3]:
# From COVID-19 API
# https://covid19api.com/

#base_url = 'https://api.covid19api.com/summary'
#headers = {"X-Access-Token":covid_api}

#response = requests.get(base_url, headers=headers).json()
#print(json.dumps(response, indent=4, sort_keys=True))
#response['Countries']

In [6]:
# -----------------------
# Get risk factor data for North Carolina, based on census data
# THIS IS WORKING PROPERLY
# -----------------------

# Import CRE csv from
# https://www2.census.gov/data/experimental-data-products/community-resilience-estimates/2020/

#risk_path = "https://www2.census.gov/data/experimental-data-products/community-resilience-estimates/2020/cre-2018-a11.csv"
# If the above breaks, use the downloaded CSV
risk_path = "data/cre-2018-a11.csv"
risk_df = pd.read_csv(risk_path)

# Break down by North Carolina
nc_risk_df = risk_df.loc[risk_df['STABREV']=='NC']

# Display dataframe
nc_risk_df


,state,county,tract,rfgrp,prednum,prednum_moe,predrt,predrt_moe,stname,STABREV,ctname,popuni
150009,37,1,0,0RF,48284,9880.2,29.84,6.11,North Carolina,NC,"Alamance County, NC",161790
150010,37,1,0,1-2RF,74707,10852.5,46.18,6.71,North Carolina,NC,"Alamance County, NC",161790
150011,37,1,0,3PLRF,38799,9176.3,23.98,5.67,North Carolina,NC,"Alamance County, NC",161790
150012,37,1,20100,0RF,1122,385.8,26.26,9.03,North Carolina,NC,"Alamance County, NC",4272
150013,37,1,20100,1-2RF,2003,410.3,46.89,9.60,North Carolina,NC,"Alamance County, NC",4272
...,...,...,...,...,...,...,...,...,...,...,...,...
156889,37,199,960300,1-2RF,1289,301.9,42.19,9.88,North Carolina,NC,"Yancey County, NC",3055
156890,37,199,960300,3PLRF,827,283.9,27.07,9.29,North Carolina,NC,"Yancey County, NC",3055
156891,37,199,960400,0RF,1135,358.4,26.43,8.34,North Carolina,NC,"Yancey County, NC",4295
156892,37,199,960400,1-2RF,1998,401.6,46.52,9.35,North Carolina,NC,"Yancey County, NC",4295


In [28]:
# -----------------------
# Clean up the data
# THIS IS WORKING PROPERLY
# -----------------------
nc_risk_df = nc_risk_df[['tract','rfgrp','prednum','ctname','popuni']]
nc_risk_rename_df = nc_risk_df.rename(columns={
    'rfgrp':'risk_group',
    'prednum':'risk_pop',
    'ctname':'county',
    'popuni':'total_pop'})

nc_risk_rename_df['county'].replace(' County, NC','',regex=True,inplace=True)

# Now we only need to keep the data where tract = 0, as that's the county summary
nc_risk_grouped = nc_risk_rename_df.groupby(['county','tract','risk_group']).sum()
nc_risk_grouped.reset_index(inplace=True)
nc_risk_filtered = nc_risk_grouped[nc_risk_grouped['tract'] == 0] 

# Display the dataframe
nc_risk_filtered = nc_risk_filtered[['county','risk_group','risk_pop','total_pop']]
nc_risk_filtered

,county,risk_group,risk_pop,total_pop
0,Alamance,0RF,48284,161790
1,Alamance,1-2RF,74707,161790
2,Alamance,3PLRF,38799,161790
111,Alexander,0RF,9789,36096
112,Alexander,1-2RF,17442,36096
...,...,...,...,...
6844,Yadkin,1-2RF,16689,37498
6845,Yadkin,3PLRF,10706,37498
6867,Yancey,0RF,4307,17870
6868,Yancey,1-2RF,8271,17870


In [42]:
# -----------------------
# Reports statewide North Carolina risk factor groups
# THIS IS WORKING PROPERLY
# -----------------------

# Make a new dataframe from the other
nc_overall_risk_df = nc_risk_filtered

# Group by risk factors
nc_overall_risk = nc_overall_risk_df.groupby('risk_group').sum()

# Build new series to hold percentages
nc_overall_risk['percentage'] = ''

# Iterate over rows to calculate percentages, round, and store in the new series 
for index, row in nc_overall_risk.iterrows():
    nc_overall_risk.at[index,'percentage'] = round((row['risk_pop'] / row['total_pop'])*100,2)

# Display the dataframe
nc_overall_risk

,risk_pop,total_pop,percentage
risk_group,,,
0RF,2896542,10219101,28.34
1-2RF,4866961,10219101,47.63
3PLRF,2455598,10219101,24.03


In [45]:
# -----------------------
# Reports fisk factor groups, by county, for North Carolina, based on census data
# THIS IS WORKING PROPERLY
# -----------------------

nc_county_risk = nc_risk_filtered

# Build new series to hold percentages
nc_county_risk['percentage'] = ''

# Iterate over rows to calculate percentages, round, and store in the new series 
for index, row in nc_county_risk.iterrows():
    nc_county_risk.at[index,'percentage'] = round((row['risk_pop'] / row['total_pop'])*100,2)

# Display the dataframe
nc_county_risk


,county,risk_group,risk_pop,total_pop,percentage
0,Alamance,0RF,48284,161790,29.84
1,Alamance,1-2RF,74707,161790,46.18
2,Alamance,3PLRF,38799,161790,23.98
111,Alexander,0RF,9789,36096,27.12
112,Alexander,1-2RF,17442,36096,48.32
...,...,...,...,...,...
6844,Yadkin,1-2RF,16689,37498,44.51
6845,Yadkin,3PLRF,10706,37498,28.55
6867,Yancey,0RF,4307,17870,24.1
6868,Yancey,1-2RF,8271,17870,46.28


In [43]:
# -----------------------
# This is a NYTimes report on national COVID-19 cases and deaths, sorted by county
# From: https://github.com/nytimes/covid-19-data (us-counties.csv)
# THIS IS WORKING PROPERLY
# -----------------------

# Pull in the live data
counties_path = "https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv"
# If the above breaks, we can refer to the saved version of the csv, below:
#counties_path = "data/us-counties.csv"

# Save to a dataframe
covid_df = pd.read_csv(counties_path)

# Sort for NC
covid_county_df = covid_df.loc[covid_df['state']=='North Carolina']

# Drop irrelevant column(s)
covid_county_clean_df = covid_county_df.drop(['fips'], 1)

# Grab last 100 rows (for 100 counties to get latest data)
covid_county_latest_df = covid_county_clean_df.tail(100)

# Diplay the dataframe
covid_county_latest_df

,date,county,state,cases,deaths
345831,2020-07-18,Alamance,North Carolina,1648,39
345832,2020-07-18,Alexander,North Carolina,210,1
345833,2020-07-18,Alleghany,North Carolina,51,0
345834,2020-07-18,Anson,North Carolina,253,1
345835,2020-07-18,Ashe,North Carolina,73,1
...,...,...,...,...,...
345926,2020-07-18,Wayne,North Carolina,2082,34
345927,2020-07-18,Wilkes,North Carolina,653,7
345928,2020-07-18,Wilson,North Carolina,1099,28
345929,2020-07-18,Yadkin,North Carolina,404,6


In [44]:
# -----------------------
# Build a dataframe that shows worst 10 counties in cases and deaths
# THIS IS WORKING PROPERLY
# -----------------------

# Sort by cases, highest to lowest
sorted_cases = covid_county_latest_df.sort_values(by=['cases'],ascending=False).reset_index()
# Sort by deaths, highest to lowest
sorted_deaths = covid_county_latest_df.sort_values(by=['deaths'],ascending=False).reset_index()

# Build a new dataframe to hold top counties and the value, sorted by cases and deaths
top_df = pd.DataFrame(columns=['Most cases','Case count','Most deaths','Death count'])

# Add the cases to the new dataframe
top_df['Most cases'] = sorted_cases['county'].values
top_df['Case count'] = sorted_cases['cases'].values

# Add the deaths to the new dataframe
top_df['Most deaths'] = sorted_deaths['county'].values
top_df['Death count'] = sorted_deaths['deaths'].values

# Show the top 10 counties for cases and deaths
top_df.head(10)

,Most cases,Case count,Most deaths,Death count
0,Mecklenburg,17047,Mecklenburg,177
1,Wake,8668,Guilford,133
2,Durham,5011,Wake,89
3,Forsyth,4122,Durham,72
4,Guilford,4061,Henderson,52
5,Johnston,2349,Buncombe,50
6,Gaston,2189,Chatham,47
7,Union,2145,Robeson,46
8,Wayne,2082,Cumberland,46
9,Robeson,1928,Rowan,45


In [47]:
# -----------------------
# Combine dataframes from NYTimes data and CRE data
# NYT = covid_county_latest_df
# CRE = nc_county_risk
# THIS IS WORKING PROPERLY
# -----------------------

# Do the merge
combined_data = pd.merge(nc_county_risk, covid_county_latest_df, on='county', how='outer')

# Display the dataframe
combined_data


,county,risk_group,risk_pop,total_pop,percentage,date,state,cases,deaths
0,Alamance,0RF,48284,161790,29.84,2020-07-18,North Carolina,1648,39
1,Alamance,1-2RF,74707,161790,46.18,2020-07-18,North Carolina,1648,39
2,Alamance,3PLRF,38799,161790,23.98,2020-07-18,North Carolina,1648,39
3,Alexander,0RF,9789,36096,27.12,2020-07-18,North Carolina,210,1
4,Alexander,1-2RF,17442,36096,48.32,2020-07-18,North Carolina,210,1
...,...,...,...,...,...,...,...,...,...
295,Yadkin,1-2RF,16689,37498,44.51,2020-07-18,North Carolina,404,6
296,Yadkin,3PLRF,10706,37498,28.55,2020-07-18,North Carolina,404,6
297,Yancey,0RF,4307,17870,24.1,2020-07-18,North Carolina,66,0
298,Yancey,1-2RF,8271,17870,46.28,2020-07-18,North Carolina,66,0


In [67]:
# -----------------------
# Summary that compares 2 factors from above dataframe 
# -----------------------

county = ''
for i, row in combined_data.iterrows():
    if county == row[0]:
        pass
    else:
        print(f'{row[0]} county | cases by total population: {round((row[7]/row[3])*100,2)}%')
        print(f'{row[0]} county | deaths by total population: {round((row[8]/row[3])*100,2)}%')
        print(f'{row[0]} county | deaths by cases: {round((row[8]/row[7])*100,2)}%')
        print('----')
        county = row[0]
        #if row[1] != '0RF':
        #    print(f'{row[0]} county | deaths by cases: {round((row[8]/row[7])*100,2)}%')
        
        
        
    




Alamance county | cases by total population: 1.02%
Alamance county | deaths by total population: 0.02%
Alamance county | deaths by cases: 2.37%
----
Alexander county | cases by total population: 0.58%
Alexander county | deaths by total population: 0.0%
Alexander county | deaths by cases: 0.48%
----
Alleghany county | cases by total population: 0.46%
Alleghany county | deaths by total population: 0.0%
Alleghany county | deaths by cases: 0.0%
----
Anson county | cases by total population: 1.1%
Anson county | deaths by total population: 0.0%
Anson county | deaths by cases: 0.4%
----
Ashe county | cases by total population: 0.27%
Ashe county | deaths by total population: 0.0%
Ashe county | deaths by cases: 1.37%
----
Avery county | cases by total population: 0.27%
Avery county | deaths by total population: 0.0%
Avery county | deaths by cases: 0.0%
----
Beaufort county | cases by total population: 0.4%
Beaufort county | deaths by total population: 0.0%
Beaufort county | deaths by cases: 0.0

In [ ]:
# -----------------------
# COVID testing location data
# From https://github.com/codersagainstcovidorg/covid19testing-backend
# -----------------------

testing_df = pd.DataFrame(columns=['Name','Lat','Lon','City','State','Collecting','Evaluating'])
testing_df

base_url = "http://api.findcovidtesting.com/api/v1/location"
response = requests.get(base_url).json()

In [ ]:
# -----------------------
# Use this to see how the response is structured:
#response[0]

# -----------------------
# Relevant info seems to be:
#     location_name
#     is_collecting_samples - boolean
#     is_evaluating_symptoms - boolean
#     location_address_locality = city
#     location_address_region = state
#     location_latitude
#     location_longitude


In [ ]:
# Iterate through the response and store selected variables in dataframe

for i in range(len(response)):
    if response[i]['location_address_region'] == 'NC':
        testing_df.loc[i,'Name'] = response[i]['location_name']
        testing_df.loc[i,'Lat'] = response[i]['location_latitude']
        testing_df.loc[i,'Lon'] = response[i]['location_longitude']
        testing_df.loc[i,'City'] = response[i]['location_address_locality']
        testing_df.loc[i,'State'] = response[i]['location_address_region']
        testing_df.loc[i,'Collecting'] = response[i]['is_collecting_samples']
        testing_df.loc[i,'Evaluating'] = response[i]['is_evaluating_symptoms']

# Display the dataframe       
testing_df

In [ ]:
gmaps.configure(api_key=google_api)

# Build the map
# Set center and zoom level
fig = gmaps.figure(map_type='HYBRID',center=(35.3,-79.5), zoom_level=7)
locations = testing_df[['Lat', 'Lon']]
hover = testing_df['Name'].tolist()

symbols = gmaps.symbol_layer(locations, fill_color='white', stroke_color='white', scale = 2, info_box_content = hover)
fig.add_layer(symbols)


# Display the map
fig